In [73]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
import pandas as pd
import re, os
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk.corpus
import spacy
nltk.download("stopwords")
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [75]:
os.chdir('/content/drive/MyDrive/')

In [76]:
df = pd.read_csv('twitter_df_may14.csv')
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Tweet User Id,Tweet Text,Tweet Time,Tweet User Name,Twitter @ Name,Tweet User Location,Tweet Coordinates,Place Info,Language
0,1.22598E+18,@caleb_efird I had to go see for myself and.....it gets worse https://t.co/iEBc15mla7,14-05-2021 23:59,Hayden Flood,flood_hayden,NaN,NaN,NaN,en
1,1.28135E+18,if it in the ground it will grow in food waste is not staying at Waste plants flood water move germs from waste plants but we can kills germs in waste before we put waste in the ground we have the meansCovid-19: Sewage testing ramped up in England https://t.co/tp3yQjRbPX,21-05-2021 23:59,Robert McFarland,RobertM06666153,"Marion, SC",NaN,"Marion, SC",en
2,3362182379,Monitoring the local river levels in Braithwaite for the flood group on 22 May 2021 00:59 https://t.co/lkxj41LKGF,21-05-2021 23:59,CA125SY,CA125SY,"Braithwaite, England",NaN,NaN,en
3,2418108972,all the water leaving my brain to go flood my puss--,21-05-2021 23:59,嗚呼☆,GaikotsuRamen,🇨🇵,NaN,NaN,en
4,2531338800,@ozthegoatguy Are the goats expecting a flood? They are all headed for the high ground?,21-05-2021 23:58,Old Man Dan,notsoyounggrunt,"Durham, NC",NaN,NaN,en


In [77]:
df.shape

(82213, 9)

In [78]:
df.isnull().sum()

Tweet User Id              1
Tweet Text                 1
Tweet Time                 2
Tweet User Name            4
Twitter @ Name             2
Tweet User Location    20383
Tweet Coordinates      81020
Place Info             79447
Language                   4
dtype: int64

In [79]:
pd.set_option('display.max_colwidth', None)

In [80]:
mod_df = df.dropna( how='any',
                    subset=['Tweet Text'])
mod_df.isnull().sum()

Tweet User Id              0
Tweet Text                 0
Tweet Time                 1
Tweet User Name            4
Twitter @ Name             1
Tweet User Location    20382
Tweet Coordinates      81019
Place Info             79446
Language                   3
dtype: int64

In [81]:
mod_df.shape

(82212, 9)

Below I have removed 'https' links and username defined by @, as links are not useful, also username won't help much in the detection of location. Whole purpose of doing this is to detect the locations from the tweets. My purpose of not removing the hastags is, as in many of the hastags we can get the location names, so I have kept it there.

In [82]:
def cleaned_data(text):
    clean=re.sub("http\S+","",text)
    clean = clean.lower()
    clean=re.sub("@\S+","",clean)
    clean = re.sub('#', '', clean)
    clean = re.sub('&amp', '', clean)
    clean = re.sub(':', '', clean)
    clean = re.sub('/', '', clean)
    clean = re.sub(r'\?{2,}', '?', clean)
    clean = re.sub(r'\!{2,}', '!', clean)
    clean = re.sub(r'\.{2,}', '.', clean)
    # clean = re.sub('!!\S+', '!', clean)
    clean = re.sub(';', '', clean)
    clean = re.sub(r'[(]', '', clean)
    clean = re.sub(r'[)]', '', clean)
    clean = re.sub('-', ' ', clean)
    clean=re.sub('"'," ",clean)
    clean=re.sub("\s{2,}"," ",clean)
    clean=clean.strip()
    return clean
mod_df['Cleaned Tweets'] = mod_df['Tweet Text'].apply(cleaned_data)
mod_df.head(40)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Tweet User Id,Tweet Text,Tweet Time,Tweet User Name,Twitter @ Name,Tweet User Location,Tweet Coordinates,Place Info,Language,Cleaned Tweets
0,1.22598E+18,@caleb_efird I had to go see for myself and.....it gets worse https://t.co/iEBc15mla7,14-05-2021 23:59,Hayden Flood,flood_hayden,NaN,NaN,NaN,en,i had to go see for myself and.it gets worse
1,1.28135E+18,if it in the ground it will grow in food waste is not staying at Waste plants flood water move germs from waste plants but we can kills germs in waste before we put waste in the ground we have the meansCovid-19: Sewage testing ramped up in England https://t.co/tp3yQjRbPX,21-05-2021 23:59,Robert McFarland,RobertM06666153,"Marion, SC",NaN,"Marion, SC",en,if it in the ground it will grow in food waste is not staying at waste plants flood water move germs from waste plants but we can kills germs in waste before we put waste in the ground we have the meanscovid 19 sewage testing ramped up in england
2,3362182379,Monitoring the local river levels in Braithwaite for the flood group on 22 May 2021 00:59 https://t.co/lkxj41LKGF,21-05-2021 23:59,CA125SY,CA125SY,"Braithwaite, England",NaN,NaN,en,monitoring the local river levels in braithwaite for the flood group on 22 may 2021 0059
3,2418108972,all the water leaving my brain to go flood my puss--,21-05-2021 23:59,嗚呼☆,GaikotsuRamen,🇨🇵,NaN,NaN,en,all the water leaving my brain to go flood my puss
4,2531338800,@ozthegoatguy Are the goats expecting a flood? They are all headed for the high ground?,21-05-2021 23:58,Old Man Dan,notsoyounggrunt,"Durham, NC",NaN,NaN,en,are the goats expecting a flood? they are all headed for the high ground?
5,3362182379,Monitoring the local river levels in Braithwaite for the flood group on 22 May 2021 00:58 https://t.co/7IGtovynZr,21-05-2021 23:58,CA125SY,CA125SY,"Braithwaite, England",NaN,NaN,en,monitoring the local river levels in braithwaite for the flood group on 22 may 2021 0058
6,1.29919E+18,@sevenlovelyhope IM AWAKE NOW HELLO YOURE SO CUTE IM GOING TO FLOOD YOUR TL AGAIN NOW,21-05-2021 23:58,ENE LOVES JUNGKOOK,nochickenkoo,NaN,NaN,NaN,en,im awake now hello youre so cute im going to flood your tl again now
7,8.74324E+17,"@popeyeotaku Like there is the time those cattle rustlers in movies tried to flood them to death in a dead volcano, the time the ghost of the Red Baron tried to mow them down with plane propellers, that Clown in where are you, Charlie the robot could've destroyed them with one tackle",21-05-2021 23:58,Triblast,Triblast28,NaN,NaN,NaN,en,"like there is the time those cattle rustlers in movies tried to flood them to death in a dead volcano, the time the ghost of the red baron tried to mow them down with plane propellers, that clown in where are you, charlie the robot could've destroyed them with one tackle"
8,1.33266E+18,@Embarassclarto Naps during the day are a dangerous play for me if the flood gates open i can sleep from 3pm to 10pm 😭,21-05-2021 23:57,🦉,FAKEJODl,NaN,NaN,NaN,en,naps during the day are a dangerous play for me if the flood gates open i can sleep from 3pm to 10pm 😭
9,1.09874E+18,@HarrisRichard77 I think he’s peed off by the flood of criticism from #PAKISTANIS on his timeline after they saw the clip in which he try’s to grill #ShireenMazari over the issue of Ughur Muslims but doesn’t really get much out of it.,21-05-2021 23:56,noor aawan,noor_aawan,"Manchester, England",NaN,NaN,en,i think he’s peed off by the flood of criticism from pakistanis on his timeline after they saw the clip in which he try’s to grill shireenmazari over the issue of ughur muslims but doesn’t really get much out of it.


In [83]:
!pip install contractions

In [84]:
import contractions

**Below python package helps in expand contractions in text like don't -> do not, isn't -> is not etc..**

In [85]:
text = '''I'll be there within 5 min. Shouldn't you be there too? 
          I'd love to see u there my dear. It's awesome to meet new friends.
          We've been waiting for this day for so long.'''
  
# creating an empty list
expanded_words = []    
for word in text.split():
  # using contractions.fix to expand the shotened words
  expanded_words.append(contractions.fix(word))   
    
expanded_text = ' '.join(expanded_words)
print('Original text: ' + text)
print('Expanded_text: ' + expanded_text)

Original text: I'll be there within 5 min. Shouldn't you be there too? 
          I'd love to see u there my dear. It's awesome to meet new friends.
          We've been waiting for this day for so long.
Expanded_text: I will be there within 5 min. should not you be there too? I would love to see you there my dear. it is awesome to meet new friends. we have been waiting for this day for so long.


In [86]:
def expanded(text):
    expanded_words = []    
    try:
      for word in text.split():
        expanded_words.append(contractions.fix(word))
    except Exception:
      pass         
    expanded_text = ' '.join(expanded_words)
    return expanded_text

mod_df['Cleaned Tweets'] = mod_df['Cleaned Tweets'].apply(expanded)
mod_df.head(15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,Tweet User Id,Tweet Text,Tweet Time,Tweet User Name,Twitter @ Name,Tweet User Location,Tweet Coordinates,Place Info,Language,Cleaned Tweets
0,1.22598E+18,@caleb_efird I had to go see for myself and.....it gets worse https://t.co/iEBc15mla7,14-05-2021 23:59,Hayden Flood,flood_hayden,NaN,NaN,NaN,en,i had to go see for myself and.it gets worse
1,1.28135E+18,if it in the ground it will grow in food waste is not staying at Waste plants flood water move germs from waste plants but we can kills germs in waste before we put waste in the ground we have the meansCovid-19: Sewage testing ramped up in England https://t.co/tp3yQjRbPX,21-05-2021 23:59,Robert McFarland,RobertM06666153,"Marion, SC",NaN,"Marion, SC",en,if it in the ground it will grow in food waste is not staying at waste plants flood water move germs from waste plants but we can kills germs in waste before we put waste in the ground we have the meanscovid 19 sewage testing ramped up in england
2,3362182379,Monitoring the local river levels in Braithwaite for the flood group on 22 May 2021 00:59 https://t.co/lkxj41LKGF,21-05-2021 23:59,CA125SY,CA125SY,"Braithwaite, England",NaN,NaN,en,monitoring the local river levels in braithwaite for the flood group on 22 may 2021 0059
3,2418108972,all the water leaving my brain to go flood my puss--,21-05-2021 23:59,嗚呼☆,GaikotsuRamen,🇨🇵,NaN,NaN,en,all the water leaving my brain to go flood my puss
4,2531338800,@ozthegoatguy Are the goats expecting a flood? They are all headed for the high ground?,21-05-2021 23:58,Old Man Dan,notsoyounggrunt,"Durham, NC",NaN,NaN,en,are the goats expecting a flood? they are all headed for the high ground?
5,3362182379,Monitoring the local river levels in Braithwaite for the flood group on 22 May 2021 00:58 https://t.co/7IGtovynZr,21-05-2021 23:58,CA125SY,CA125SY,"Braithwaite, England",NaN,NaN,en,monitoring the local river levels in braithwaite for the flood group on 22 may 2021 0058
6,1.29919E+18,@sevenlovelyhope IM AWAKE NOW HELLO YOURE SO CUTE IM GOING TO FLOOD YOUR TL AGAIN NOW,21-05-2021 23:58,ENE LOVES JUNGKOOK,nochickenkoo,NaN,NaN,NaN,en,I am awake now hello you are so cute I am going to flood your tl again now
7,8.74324E+17,"@popeyeotaku Like there is the time those cattle rustlers in movies tried to flood them to death in a dead volcano, the time the ghost of the Red Baron tried to mow them down with plane propellers, that Clown in where are you, Charlie the robot could've destroyed them with one tackle",21-05-2021 23:58,Triblast,Triblast28,NaN,NaN,NaN,en,"like there is the time those cattle rustlers in movies tried to flood them to death in a dead volcano, the time the ghost of the red baron tried to mow them down with plane propellers, that clown in where are you, charlie the robot could have destroyed them with one tackle"
8,1.33266E+18,@Embarassclarto Naps during the day are a dangerous play for me if the flood gates open i can sleep from 3pm to 10pm 😭,21-05-2021 23:57,🦉,FAKEJODl,NaN,NaN,NaN,en,naps during the day are a dangerous play for me if the flood gates open i can sleep from 3pm to 10pm 😭
9,1.09874E+18,@HarrisRichard77 I think he’s peed off by the flood of criticism from #PAKISTANIS on his timeline after they saw the clip in which he try’s to grill #ShireenMazari over the issue of Ughur Muslims but doesn’t really get much out of it.,21-05-2021 23:56,noor aawan,noor_aawan,"Manchester, England",NaN,NaN,en,i think he is peed off by the flood of criticism from pakistanis on his timeline after they saw the clip in which he try’s to grill shireenmazari over the issue of ughur muslims but does not really get much out of it.


Below function is for removing emojis from the text.

In [92]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

mod_df['cleaned_emoji_removed'] = mod_df['Cleaned Tweets'].apply(remove_emoji)
mod_df.head(40)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Tweet User Id,Tweet Text,Tweet Time,Tweet User Name,Twitter @ Name,Tweet User Location,Tweet Coordinates,Place Info,Language,Cleaned Tweets,cleaned_emoji_removed
0,1.22598E+18,@caleb_efird I had to go see for myself and.....it gets worse https://t.co/iEBc15mla7,14-05-2021 23:59,Hayden Flood,flood_hayden,NaN,NaN,NaN,en,i had to go see for myself and.it gets worse,i had to go see for myself and.it gets worse
1,1.28135E+18,if it in the ground it will grow in food waste is not staying at Waste plants flood water move germs from waste plants but we can kills germs in waste before we put waste in the ground we have the meansCovid-19: Sewage testing ramped up in England https://t.co/tp3yQjRbPX,21-05-2021 23:59,Robert McFarland,RobertM06666153,"Marion, SC",NaN,"Marion, SC",en,if it in the ground it will grow in food waste is not staying at waste plants flood water move germs from waste plants but we can kills germs in waste before we put waste in the ground we have the meanscovid 19 sewage testing ramped up in england,if it in the ground it will grow in food waste is not staying at waste plants flood water move germs from waste plants but we can kills germs in waste before we put waste in the ground we have the meanscovid 19 sewage testing ramped up in england
2,3362182379,Monitoring the local river levels in Braithwaite for the flood group on 22 May 2021 00:59 https://t.co/lkxj41LKGF,21-05-2021 23:59,CA125SY,CA125SY,"Braithwaite, England",NaN,NaN,en,monitoring the local river levels in braithwaite for the flood group on 22 may 2021 0059,monitoring the local river levels in braithwaite for the flood group on 22 may 2021 0059
3,2418108972,all the water leaving my brain to go flood my puss--,21-05-2021 23:59,嗚呼☆,GaikotsuRamen,🇨🇵,NaN,NaN,en,all the water leaving my brain to go flood my puss,all the water leaving my brain to go flood my puss
4,2531338800,@ozthegoatguy Are the goats expecting a flood? They are all headed for the high ground?,21-05-2021 23:58,Old Man Dan,notsoyounggrunt,"Durham, NC",NaN,NaN,en,are the goats expecting a flood? they are all headed for the high ground?,are the goats expecting a flood? they are all headed for the high ground?
5,3362182379,Monitoring the local river levels in Braithwaite for the flood group on 22 May 2021 00:58 https://t.co/7IGtovynZr,21-05-2021 23:58,CA125SY,CA125SY,"Braithwaite, England",NaN,NaN,en,monitoring the local river levels in braithwaite for the flood group on 22 may 2021 0058,monitoring the local river levels in braithwaite for the flood group on 22 may 2021 0058
6,1.29919E+18,@sevenlovelyhope IM AWAKE NOW HELLO YOURE SO CUTE IM GOING TO FLOOD YOUR TL AGAIN NOW,21-05-2021 23:58,ENE LOVES JUNGKOOK,nochickenkoo,NaN,NaN,NaN,en,I am awake now hello you are so cute I am going to flood your tl again now,I am awake now hello you are so cute I am going to flood your tl again now
7,8.74324E+17,"@popeyeotaku Like there is the time those cattle rustlers in movies tried to flood them to death in a dead volcano, the time the ghost of the Red Baron tried to mow them down with plane propellers, that Clown in where are you, Charlie the robot could've destroyed them with one tackle",21-05-2021 23:58,Triblast,Triblast28,NaN,NaN,NaN,en,"like there is the time those cattle rustlers in movies tried to flood them to death in a dead volcano, the time the ghost of the red baron tried to mow them down with plane propellers, that clown in where are you, charlie the robot could have destroyed them with one tackle","like there is the time those cattle rustlers in movies tried to flood them to death in a dead volcano, the time the ghost of the red baron tried to mow them down with plane propellers, that clown in where are you, charlie the robot could have destroyed them with one tackle"
8,1.33266E+18,@Embarassclarto Naps during the day are a dangerous play for me if the flood gates open i can sleep from 3pm to 10pm 😭,21-05-2021 23:57,🦉,FAKEJODl,NaN,NaN,NaN,en,naps during the day are a dangerous play for me if t

In [88]:
mod_df.to_csv('preprocessed_twitter_df_may14.csv', index = False)

**Almost all most of the pre-processing is done, now the only part that is remaining is extracting the locations from all the text. I will update it soon. You guys also work on it.**

In [89]:
nlp = spacy.load('en_core_web_sm')

In [90]:
# def entity_extract(text):
#   doc = nlp(text)
#   return [ent.text for ent in doc.ents if ent.label_ == 'GPE']

# df2['Location Name'] = df2['Cleaned Tweets Further'].apply(entity_extract)

# df2.head()


In [91]:
a = 'this is a robberry ???????????? and we are doing it'
clean = re.sub(r'\?{2,}', '?', a)
print(clean)

this is a robberry ? and we are doing it
